# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
# What is the sum of the outputs of the generator for limit = 5 ?

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

total = 0
for sqrt_value in generator:
    print(sqrt_value)
    total += sqrt_value
    print('sum of outputs:', total)


1.0
sum of outputs: 1.0
1.4142135623730951
sum of outputs: 2.414213562373095
1.7320508075688772
sum of outputs: 4.146264369941973
2.0
sum of outputs: 6.146264369941973
2.23606797749979
sum of outputs: 8.382332347441762


In [2]:
# What is the 13th number yielded by the generator?
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
# generators
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [4]:
# Install required libraries, if needed
!pip install -Uq dlt "dlt[duckdb]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00


In [6]:
# Data loading - Load the first generator
import dlt
import duckdb

pipeline = dlt.pipeline(
    pipeline_name="load_people_data", destination="duckdb",
                                    dataset_name="people_data")

load_info = pipeline.run(data=people_1(), table_name="people_append",  primary_key="Name")

print(load_info)

# show outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_append table below:")

folks = conn.sql("SELECT * FROM people_append").df()
display(folks)

# sum of anges of all people
query = "SELECT SUM(age) AS sum_ages FROM people_append"

sum_ages = conn.sql(query).df()
display(sum_ages)

Pipeline load_people_data load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////content/load_people_data.duckdb location to store data
Load package 1707794654.4698484 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_append       │
└─────────────────────┘




 people_append table below:


,name,id,age,city,_dlt_load_id,_dlt_id
0,Person_1,1,26,City_A,1707794654.4698484,wixDp4Aep70woQ
1,Person_2,2,27,City_A,1707794654.4698484,EL3Yk7fsJ+ZSKQ
2,Person_3,3,28,City_A,1707794654.4698484,FE4iDsSy4eKz0Q
3,Person_4,4,29,City_A,1707794654.4698484,4zO+ymEuE7rNYw
4,Person_5,5,30,City_A,1707794654.4698484,VXUR76FrY1gTNA


,sum_ages
0,140.0


In [7]:
# incremental loading - append and check output
load_info_2 = pipeline.run(data=people_2(), table_name="people_append",
                           write_disposition="append",
                           primary_key="Name")

print(load_info_2)

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_append table below:")

folks = conn.sql("SELECT * FROM people_append").df()
display(folks)

# sum of anges of all people
query = "SELECT SUM(age) AS sum_ages FROM people_append"

sum_ages = conn.sql(query).df()
display(sum_ages)

Pipeline load_people_data load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////content/load_people_data.duckdb location to store data
Load package 1707794666.3367517 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_append       │
└─────────────────────┘




 people_append table below:


,name,id,age,city,_dlt_load_id,_dlt_id,occupation
0,Person_1,1,26,City_A,1707794654.4698484,wixDp4Aep70woQ,None
1,Person_2,2,27,City_A,1707794654.4698484,EL3Yk7fsJ+ZSKQ,None
2,Person_3,3,28,City_A,1707794654.4698484,FE4iDsSy4eKz0Q,None
3,Person_4,4,29,City_A,1707794654.4698484,4zO+ymEuE7rNYw,None
4,Person_5,5,30,City_A,1707794654.4698484,VXUR76FrY1gTNA,None
5,Person_3,3,33,City_B,1707794666.3367517,4zDFLg7z5wEpig,Job_3
6,Person_4,4,34,City_B,1707794666.3367517,OU5Ty11n4jN5kw,Job_4
7,Person_5,5,35,City_B,1707794666.3367517,dTDWIh8OHLzGoQ,Job_5
8,Person_6,6,36,City_B,1707794666.3367517,yBHlCJvW2g7ZCA,Job_6
9,Person_7,7,37,City_B,1707794666.3367517,U8kUUmITElMnkg,Job_7


,sum_ages
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [8]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [9]:
# to do: homework :)
import dlt
import duckdb

pipeline = dlt.pipeline(
    pipeline_name="load_people_data", destination="duckdb", dataset_name="people_data"
)
load_info = pipeline.run(data=people_1(), table_name="people_merge",
                         primary_key="ID")

print(load_info)


# show outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_merge table below:")

folks = conn.sql("SELECT * FROM people_merge").df()
display(folks)

# sum of anges of all people
query = "SELECT SUM(age) AS sum_ages FROM people_merge"

sum_ages = conn.sql(query).df()
display(sum_ages)

Pipeline load_people_data load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////content/load_people_data.duckdb location to store data
Load package 1707794682.4159036 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_append       │
│ people_merge        │
└─────────────────────┘




 people_merge table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707794682.4159036,sd4lV4bp+6Odqw
1,2,Person_2,27,City_A,1707794682.4159036,hCFi+V4YjI3NIw
2,3,Person_3,28,City_A,1707794682.4159036,ZlKkI6BBRX6Oxw
3,4,Person_4,29,City_A,1707794682.4159036,Vu1gYd9qNxmn1g
4,5,Person_5,30,City_A,1707794682.4159036,ObqCtvqxMmVwGw


,sum_ages
0,140.0


In [10]:
# incremental loading - merge and check output
load_info_3 = pipeline.run(data=people_2(), table_name="people_merge",
                           write_disposition="merge",
                           primary_key="ID")

print(load_info_3)

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_merge table below:")

folks = conn.sql("SELECT * FROM people_merge").df()
display(folks)

# sum of anges of all people
query = "SELECT SUM(age) AS sum_ages FROM people_merge"

sum_ages = conn.sql(query).df()
display(sum_ages)

Pipeline load_people_data load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////content/load_people_data.duckdb location to store data
Load package 1707794696.8535113 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_append       │
│ people_merge        │
└─────────────────────┘




 people_merge table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707794682.4159036,sd4lV4bp+6Odqw,None
1,2,Person_2,27,City_A,1707794682.4159036,hCFi+V4YjI3NIw,None
2,8,Person_8,38,City_B,1707794696.8535113,w3/6MzkJ1uMSnw,Job_8
3,4,Person_4,34,City_B,1707794696.8535113,ALb+UZJ59d0O2A,Job_4
4,5,Person_5,35,City_B,1707794696.8535113,PoS5wfm+49l3bw,Job_5
5,7,Person_7,37,City_B,1707794696.8535113,kOazLaF/E4i8bQ,Job_7
6,3,Person_3,33,City_B,1707794696.8535113,bzuApLQapnwdpg,Job_3
7,6,Person_6,36,City_B,1707794696.8535113,bipvhGkYFf7msQ,Job_6


,sum_ages
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX